<a href="https://colab.research.google.com/github/lokeaichirou/LLM/blob/main/workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install Agently==3.3.4.5
!pip install mermaid-python
!pip installnest_asyncio

ERROR: unknown command "installnest_asyncio"


In [ ]:
# 使用nest_asyncio确保异步稳定性
import nest_asyncio
nest_asyncio.apply()

In [ ]:
deep_seek_url           = "https://api.deepseek.com"
deep_seek_api_key       = "sk-104cf02ecfbf41dbac6736e2bda05d26"
deep_seek_default_model = 'deepseek-chat'

In [ ]:
import json
import Agently
import os

[WARNING][2024-10-13 22:28:17.772] redis_rate_limiter.py:21 [t:137552259227648]: no redis installed, RedisRateLimiter unavailable


## 创建agent与workflow

In [ ]:
# 创建两个工作流：主工作流和分块创作工作流
main_workflow  = Agently.Workflow()
block_workflow = Agently.Workflow()

In [ ]:
# 创建agent

analyst_1 = (Agently.create_agent()
                    .set_settings("current_model", "OAIClient")
                    .set_settings("model.OAIClient.url", deep_seek_url)
                    .set_settings("model.OAIClient.auth", {"api_key": deep_seek_api_key})
                    .set_settings("model.OAIClient.options", {"model": deep_seek_default_model })
             )

analyst_2 = (Agently.create_agent()
                    .set_settings("current_model", "OAIClient")
                    .set_settings("model.OAIClient.url", deep_seek_url)
                    .set_settings("model.OAIClient.auth", {"api_key": deep_seek_api_key})
                    .set_settings("model.OAIClient.options", {"model": deep_seek_default_model })
             )

analyst_3 = (Agently.create_agent()
                    .set_settings("current_model", "OAIClient")
                    .set_settings("model.OAIClient.url", deep_seek_url)
                    .set_settings("model.OAIClient.auth", {"api_key": deep_seek_api_key})
                    .set_settings("model.OAIClient.options", {"model": deep_seek_default_model })
             )

analyst_4 = (Agently.create_agent()
                    .set_settings("current_model", "OAIClient")
                    .set_settings("model.OAIClient.url", deep_seek_url)
                    .set_settings("model.OAIClient.auth", {"api_key": deep_seek_api_key})
                    .set_settings("model.OAIClient.options", {"model": deep_seek_default_model })
             )

## 定义主工作流的工作块

In [ ]:
## 输入一句话描述

@main_workflow.chunk()
def input_match_basic_info(inputs, storage):

    storage.set("refree_name", input("[💡请输入主裁判名字]:"))

    storage.set("competition_name", input("[💡请输入赛事名称]:"))
    storage.set("home_team", input("[💡请输入主场球队]:"))
    storage.set("away_team", input("[💡请输入客场球队]:"))
    storage.set("weather_condition", input("[💡请输入天气情况]:"))

    storage.set("home_team_coach", input("[💡请输入主场球队教练]:"))
    storage.set("away_team_coach", input("[💡请输入客场球队教练]:"))

    storage.set("home_team_line_up", input("[💡请输入主场球队首发11人名单]:"))
    storage.set("home_team_substitutions", input("[💡请输入主场球队替补队员名单]:"))
    storage.set("home_team_formation", input("[💡请输入主场球队阵型]:"))

    storage.set("away_team_line_up", input("[💡请输入客场球队首发11人名单]:"))
    storage.set("away_team_substitutions", input("[💡请输入客场球队替补队员名单]:"))
    storage.set("away_team_formation", input("[💡请输入客场球队阵型]:"))

    storage.set("home_team_tactics_style", input("[💡请输入主场球队战术风格]:"))
    storage.set("away_team_tactics_style", input("[💡请输入客场球队战术风格]:"))

    storage.set("home_team_recent_results", input("[💡请输入主场球队近期战绩]:"))
    storage.set("away_team_recent_results", input("[💡请输入客场球队近期战绩]:"))

    storage.set("home_team_player_current_season_stats", input("[💡请输入主场球队球员本赛季数据]:"))
    storage.set("away_team_player_current_season_stats", input("[💡请输入客场球队球员本赛季数据]:"))

    return

In [ ]:
## 过程产出输出

@main_workflow.chunk_class()
def print_process_output(inputs, storage):

    print(f"[{ inputs['default']['title'] }]:")
    if isinstance(inputs["default"]["result"], dict):
        print(json.dumps(inputs["default"]["result"], indent=4, ensure_ascii=False))
    else:
        print(inputs["default"]["result"])
    return

In [ ]:
## 创建比赛预览

@main_workflow.chunk()
def generate_game_scenario(inputs, storage):

    confirm = "n"
    while confirm.lower() == "n":

        game_scenario       = storage.get("game_scenario", None)
        revision_suggestion = storage.get("revision_suggestion_for_game_scenario", None)

        if game_scenario and revision_suggestion:

            updated_game_scenario = (analyst_1.set_role({"角色", "你是一个足球技战术分析评论员, 预估球队整体表现"})
                                              .input({"比赛预览": game_scenario, "比赛基本情况": game_scenario["比赛基本情况"], "两队各自优势与潜在问题": game_scenario["两队各自优势与潜在问题"],
                                                      "两队主帅战术安排": game_scenario["两队主帅战术安排"], "修改建议": revision_suggestion})
                                              .instruct("""请根据{修改建议}改进{比赛基本情况}、{两队各自优势与潜在问题}、{两队主帅战术安排}并更新至{比赛预览}里的相关条目""")
                                              .output({"比赛基本情况": [{"比赛风格":   ("str", "基于双方球队和主帅风格特点给出的比赛大体风格"),
                                                                       "比赛局面":   ("str", "基于双方球队和主帅风格特点给出的比赛大体局面"),
                                                                       "比赛激烈度": ("str", "基于双方球队和主帅风格特点给出的比赛激烈程度"),}
                                                                      ],

                                                       "两队各自优势与潜在问题": [{"主队在进攻端的优势": ("str", "给出主队在进攻端的优势"),
                                                                               "主队在中场的优势":   ("str", "给出主队在中场的优势"),
                                                                               "主队在防守端的优势": ("str", "给出主队在防守端的优势"),
                                                                               "客队在进攻端的优势": ("str", "给出客队在进攻端的优势"),
                                                                               "客队在中场的优势":   ("str", "给出客队在中场的优势"),
                                                                               "客队在防守端的优势": ("str", "给出客队在防守端的优势"),
                                                                               "主队在进攻端的潜在问题与隐患": ("str", "给出主队在进攻端的潜在问题与隐患"),
                                                                               "主队在中场的潜在问题与隐患":   ("str", "给出主队在中场的潜在问题与隐患"),
                                                                               "主队在防守端的潜在问题与隐患": ("str", "给出主队在防守端的潜在问题与隐患"),
                                                                               "客队在进攻端的潜在问题与隐患": ("str", "给出客队在进攻端的潜在问题与隐患"),
                                                                               "客队在中场的潜在问题与隐患":   ("str", "给出客队在中场的潜在问题与隐患"),
                                                                               "客队在防守端的潜在问题与隐患": ("str", "给出客队在防守端的潜在问题与隐患"),}
                                                                               ],
                                                       "两队主帅战术安排": [{"主队主帅的战术安排": ("str",  "给出主队主帅的战术安排"),
                                                                          "客队主帅的战术安排": ("str", "给出客队主帅的战术安排"),}
                                                                         ]
                                                       }
                                                      ).start()
                                      )

            storage.set("game_scenario", updated_game_scenario)
            print(json.dumps(updated_game_scenario, indent=4, ensure_ascii=False))

            confirm = input("[您是否满意本次创作结果？(y/n)]: ").lower()
            if confirm.lower() == "y":
                return {"title": "比赛预览", "result": updated_game_scenario}
            else:
                storage.set("revision_suggestion_for_game_scenario", input("[请输入您的修改意见]: "))

        else:

            competition_name = storage.get("competition_name")
            home_team        = storage.get("home_team")
            away_team        = storage.get("away_team")

            home_team_coach = storage.get("home_team_coach")
            away_team_coach = storage.get("away_team_coach")

            home_team_line_up = storage.get("home_team_line_up")
            away_team_line_up = storage.get("away_team_line_up")

            home_team_substitutions = storage.get("home_team_substitutions")
            away_team_substitutions = storage.get("away_team_substitutions")

            home_team_formation = storage.get("home_team_formation")
            away_team_formation = storage.get("away_team_formation")

            home_team_tactics_style = storage.get("home_team_tactics_style")
            away_team_tactics_style = storage.get("away_team_tactics_style")

            game_scenario = (analyst_1.set_role({"角色", "你是一个足球技战术分析评论员, 预估球队整体表现",})
                                      .input({"赛事": competition_name, "主场球队": home_team, "客场球队":away_team, "主队首发": home_team_line_up, "客队首发":away_team_line_up,
                                              "主队替补": home_team_substitutions, "客队替补": away_team_substitutions, "主队阵型": home_team_formation, "客队阵型":away_team_formation,
                                              "主队主帅": home_team_coach, "客队主帅": away_team_coach, "主队技战术风格": home_team_tactics_style, "客队技战术风格": away_team_tactics_style})
                                      .instruct("""请根据{赛事}和主客场因素，并基于你对{主场球队}和{客场球队}的当前各自实力、过往技战术风格和过往恩怨的了解，结合{主队主帅}、{客队主帅}的执教风格以及本场比赛{主队技战术风格}、{客队技战术风格}，分析评估一下比赛大体风格、局面和激烈程度
                                                    同时基于你对{主队首发}、{主队替补}、{客队首发}、{客队替补}的了解和基于{主队阵型}与{客队阵型}产生的对位情况分析预估一下比赛的进一步具体情况, 体现在两队在各自攻防两端和中场的优势与潜在问题与隐患
                                                    结合上面的分析以及{主队主帅}、{客队主帅}他们的执教特点与换人习惯评估下可能的战术安排""")
                                      .output({"比赛基本情况": [{"比赛风格":   ("str", "基于双方球队和主帅风格特点给出的比赛大体风格"),
                                                               "比赛局面":   ("str", "基于双方球队和主帅风格特点给出的比赛大体局面"),
                                                               "比赛激烈度": ("str", "基于双方球队和主帅风格特点给出的比赛激烈程度"),}
                                                              ],

                                              "两队各自优势与潜在问题": [{"主队在进攻端的优势": ("str", "给出主队在进攻端的优势"),
                                                                      "主队在中场的优势":   ("str", "给出主队在中场的优势"),
                                                                      "主队在防守端的优势": ("str", "给出主队在防守端的优势"),
                                                                      "客队在进攻端的优势": ("str", "给出客队在进攻端的优势"),
                                                                      "客队在中场的优势":   ("str", "给出客队在中场的优势"),
                                                                      "客队在防守端的优势": ("str", "给出客队在防守端的优势"),
                                                                      "主队在进攻端的潜在问题与隐患": ("str", "给出主队在进攻端的潜在问题与隐患"),
                                                                      "主队在中场的潜在问题与隐患":   ("str", "给出主队在中场的潜在问题与隐患"),
                                                                      "主队在防守端的潜在问题与隐患": ("str", "给出主队在防守端的潜在问题与隐患"),
                                                                      "客队在进攻端的潜在问题与隐患": ("str", "给出客队在进攻端的潜在问题与隐患"),
                                                                      "客队在中场的潜在问题与隐患":   ("str", "给出客队在中场的潜在问题与隐患"),
                                                                      "客队在防守端的潜在问题与隐患": ("str", "给出客队在防守端的潜在问题与隐患"),}
                                                                      ],
                                              "两队主帅战术安排": [{"主队教练的战术安排": ("str",  "给出主队主帅的战术安排"),
                                                                 "客队教练的战术安排": ("str", "给出客队主帅的战术安排"),}
                                                                ]
                                              }
                                             ).start()
                              )

            storage.set("game_scenario", game_scenario)
            print(json.dumps(game_scenario, indent=4, ensure_ascii=False))

            confirm = input("[您是否满意本次创作结果？(y/n)]: ").lower()
            if confirm.lower() == "y":
                return {"title": "比赛预览", "result": game_scenario}
            else:
                storage.set("revision_suggestion_for_game_scenario", input("[请输入您的修改意见]: "))


In [ ]:
## 创建首发球员表现预估

@main_workflow.chunk()
def generate_player_performance(inputs, storage):

    confirm = "n"
    while confirm.lower() == "n":

        line_up_players_performance = storage.get("line_up_players_performance", None)
        revision_suggestion         = storage.get("revision_suggestion_for_player_performance", None)

        if line_up_players_performance and revision_suggestion:

            updated_line_up_players_performance = (analyst_2.set_role({"角色", "你是一个职业足球表现分析师, 预估球员表现"})
                                                            .input({"首发球员表现预估": line_up_players_performance,
                                                                    "主队首发球员表现": line_up_players_performance["主队首发球员表现"],
                                                                    "客队首发球员表现": line_up_players_performance["客队首发球员表现"],
                                                                    "修改建议": revision_suggestion
                                                                      }
                                                                    )
                                                            .instruct("""请根据{修改建议}改进{主队首发球员表现}、{客队首发球员表现}并更新至{首发球员表现预估}里的相关条目""")
                                                            .output({"主队首发球员表现": [{"球员姓名": ("str", ), "球风": ("str", "球员球风"), "位置": ("str", "场上位置"), "犯规属性":("str", "是否爱犯规"), "大致表现及状态": ("str", "预估场上的主要表现和对球队的贡献及问题，结合{主场球队球员本赛季数据}预判状态好坏"), "可能的亮点及问题": ("str", "结合{球风}、{位置}预估场上表现可圈可点的地方和问题")}],
                                                                     "客队首发球员表现": [{"球员姓名": ("str", ), "球风": ("str", "球员球风"), "位置": ("str", "场上位置"), "犯规属性":("str", "是否爱犯规"), "大致表现及状态": ("str", "预估场上的主要表现和对球队的贡献及问题，结合{客场球队球员本赛季数据}预判状态好坏"), "可能的亮点及问题": ("str", "结合{球风}、{位置}预估场上表现可圈可点的地方和问题")}],
                                                              }
                                                                    )
                                                            .start()
                                                    )

            storage.set("line_up_players_performance", updated_line_up_players_performance)
            print(json.dumps(updated_line_up_players_performance, indent=4, ensure_ascii=False))

            confirm = input("[您是否满意本次创作结果？(y/n)]: ").lower()
            if confirm.lower() == "y":
                return {"title": "首发球员表现预估", "result": updated_line_up_players_performance,}
            else:
                storage.set("revision_suggestion_for_player_performance", input("[请输入您的修改意见]: "))

        else:
            game_scenario      = storage.get("game_scenario")

            home_team_line_up  = storage.get("home_team_line_up")
            away_team_line_up  = storage.get("away_team_line_up")

            home_team_player_current_season_stats = storage.get("home_team_player_current_season_stats")
            away_team_player_current_season_stats = storage.get("away_team_player_current_season_stats")

            line_up_players_performance = (analyst_2.set_role({"角色", "你是一个职业足球表现分析师, 预估球员表现"})
                                                    .input({"比赛基本情况": game_scenario["比赛基本情况"], "两队各自优势与潜在问题": game_scenario["两队各自优势与潜在问题"], "两队主帅战术安排": game_scenario["两队主帅战术安排"],
                                                            "主队首发": home_team_line_up, "客队首发": away_team_line_up, "主场球队球员本赛季数据": home_team_player_current_season_stats, "客场球队球员本赛季数据": away_team_player_current_season_stats})
                                                    .instruct("""请根据{比赛基本情况}、{两队各自优势与潜在问题}和{两队主帅战术安排}, 结合你对{主队首发}、{客队首发}球员的了解以及{主场球队球员本赛季数据}、{客场球队球员本赛季数据}分析预估下场上首发球员的预期表现和状态，不要把{主场球队球员本赛季数据}、{客场球队球员本赛季数据}放在输出里""")
                                                    .output({"主队首发球员表现": [{"球员姓名": ("str", ), "球风": ("str", "球员球风"), "位置": ("str", "场上位置"), "犯规属性":("str", "是否爱犯规"), "大致表现及状态": ("str", "预估场上的主要表现和对球队的贡献及问题，结合{主场球队球员本赛季数据}预判状态好坏"), "可能的亮点及问题": ("str", "结合{球风}、{位置}预估场上表现可圈可点的地方和问题")}],
                                                             "客队首发球员表现": [{"球员姓名": ("str", ), "球风": ("str", "球员球风"), "位置": ("str", "场上位置"), "犯规属性":("str", "是否爱犯规"), "大致表现及状态": ("str", "预估场上的主要表现和对球队的贡献及问题，结合{客场球队球员本赛季数据}预判状态好坏"), "可能的亮点及问题": ("str", "结合{球风}、{位置}预估场上表现可圈可点的地方和问题")}],
                                                              }
                                                            ).start()
                                            )

            storage.set("line_up_players_performance", line_up_players_performance)
            print(json.dumps(line_up_players_performance, indent=4, ensure_ascii=False))

            confirm = input("[您是否满意本次创作结果？(y/n)]: ").lower()
            if confirm.lower() == "y":
                return {"title": "首发球员表现预估", "result": line_up_players_performance,}
            else:
                storage.set("revision_suggestion_for_player_performance", input("[请输入您的修改意见]: "))

In [ ]:
## 创建比赛情节线

@main_workflow.chunk()
def generate_game_line(inputs, storage):

    confirm = "n"
    while confirm.lower() == "n":

        game_line           = storage.get("game_line", None)
        revision_suggestion = storage.get("revision_suggestion_for_game_line", None)

        if game_line and revision_suggestion:

            updated_game_line = (analyst_3.set_role({"角色", "你是一个职业足球比赛分析预测师, 能像足球经理游戏里那样模拟推演比赛的发展"})
                                          .input({"比赛情节线": game_line, "修改建议": revision_suggestion})
                                          .instruct("""请根据{修改建议}改进{比赛情节线}并更新至{比赛情节线}里的相关条目""")
                                          .start()
                         )

            storage.set("game_line", game_line)
            print(json.dumps(game_line, indent=4, ensure_ascii=False))

            confirm = input("[您是否满意本次创作结果？(y/n)]: ").lower()
            if confirm.lower() == "y":
                return {"title": "比赛情节线", "result": updated_game_line}
            else:
                storage.set("revision_suggestion_for_game_line", input("[请输入您的修改意见]: "))

        else:
            competition_name            = storage.get("competition_name")
            refree_name                 = storage.get("refree_name")
            weather_condition           = storage.get("weather_condition")

            home_team_coach             = storage.get("home_team_coach")
            away_team_coach             = storage.get("away_team_coach")

            game_scenario               = storage.get("game_scenario")
            line_up_players_performance = storage.get("line_up_players_performance")

            game_line = (analyst_3.set_role({"角色", "你是一个职业足球比赛分析预测师, 能像足球经理游戏里那样模拟推演比赛的发展"})
                                  .input({"比赛预览": game_scenario, "首发球员表现预估": line_up_players_performance, "赛事": competition_name, "主裁判": refree_name, "天气状况": weather_condition, "主场球队教练": home_team_coach, "客场球队教练": away_team_coach})
                                  .instruct("""请根据{比赛预览}、{天气状况}、结合你对{赛事}的比赛和换人规则、{赛事}的激烈精彩程度、{主裁判}的执法风格和两支球队比赛风格与打法的了解，并围绕{首发球员表现预估}，模拟比赛发展
                                               体现在比赛大致内容及双方的态势、双方争夺的主要焦点、谁更有统治力和优势在哪里，是否有犯规， 双方主教练{主场球队教练}与{客场球队教练}可能做出的战术调整和换人调整，是否会有受伤和因为受伤导致的换人或因为战术安排发生的换人调整
                                               将比赛按每十分钟和伤停补时来切成块从而创建比赛的发展情节线""")
                                  .output({"比赛情节线":[{"该时间段内比赛情节": [{"比赛时间": ("str", "比赛时间段"),
                                                                             "比赛大致内容和走向": ("str", "比赛主要发生在球场上哪个区域，双方在该时间段内各采用什么样的比赛策略以及双方的目的，比如有的球队会抢开局，有的会采用密集防守摆大巴寻找反击机会，有的会发起进攻试探，有的会采用传控以掌握比赛，有的会采用压迫或高位逼抢来占据主动等等，根据已有信息与你对两支球队的了解来预测"),
                                                                             "双方形势": ("str", "是局面明朗还是胶着，比如互相僵持或是一方主攻势另一方主守势，等等，根据已有信息与你对两支球队的了解来预测"),
                                                                             "双方争夺的主要焦点": ("str", "例如双方侧重于争夺球权或是在进攻三区的渗透或是边路爆破或是寻找禁区内高点或是定位球的机会等等，根据已有信息与你对两支球队的了解来预测"),
                                                                             "是否均势": ("bool", "根据{双方形势}、{比赛大致内容和走向}、{双方争夺的主要焦点}判断是一方压着另一方展开进攻还是互有攻守"),
                                                                             "谁更有统治力": ("str", "如果{是否均势}是True就不用回答，None即可，如果是False，就请根据{比赛大致内容和走向}、{双方形势}、{双方争夺的主要焦点}来作答"),
                                                                             "优势方的优势": ("str", "如果{是否均势}是True就不用回答，None即可，如果是False，就请根据{谁更有统治力}、{比赛大致内容和走向}、{双方形势}、{双方争夺的主要焦点}来作答"),
                                                                             "优势方的位置": ("str", "如果{是否均势}是True就不用回答，None即可，如果是False，就请根据{优势方的优势}、{谁更有统治力}、{比赛大致内容和走向}、{双方形势}、{双方争夺的主要焦点}来回答优势方在场上什么位置具有优势"),
                                                                             "劣势方的劣势": ("str", "如果{是否均势}是True就不用回答，None即可，如果是False，就请根据{优势方的位置}、{优势方的优势}、{谁更有统治力}、{比赛大致内容和走向}、{双方形势}、{双方争夺的主要焦点}来作答"),
                                                                             "劣势方的潜在机会": ("str", "如果{是否均势}是True就不用回答，None即可，如果是False，就请根据{劣势方的劣势}、{优势方的位置}、{优势方的优势}、{谁更有统治力}、{比赛大致内容和走向}、{双方形势}、{双方争夺的主要焦点}来作答"),
                                                                              }
                                                                              ],
                                                        "主队是否有战术调整":       ("bool", "根据{该时间段内比赛情节}的场上形势及你对{主场球队教练}的执教风格的了解，判断是否会有战术调整"),
                                                        "主队会做怎样的战术调整":    ("str",  "如果{主队是否有战术调整}是False就不用回答，None即可，如果是True，就请根据{该时间段内比赛情节}的场上形势及你对{主场球队教练}的执教风格的了解给出是怎样的换人调整"),
                                                        "客队是否有战术调整":       ("bool", "根据{该时间段内比赛情节}的场上形势及你对{客场球队教练}的执教风格的了解，判断是否会有战术调整"),
                                                        "客队会做怎样的战术调整":    ("str",  "如果{客队是否有战术调整}是False就不用回答，None即可，如果是True，就请根据{该时间段内比赛情节}的场上形势及你对{客场球队教练}的执教风格的了解给出是怎样的换人调整"),
                                                        "比赛激烈程度和身体对抗强度": ("str",  "根据{该时间段内比赛情节}预测比赛激烈程度和身体对抗强度"),
                                                        "比赛犯规程度":            ("str",  "根据{比赛激烈程度和身体对抗强度}预测比赛犯规程度")
                                                          }
                                                        ]
                                            }
                                          ).start()
                                )

            storage.set("game_line", game_line)
            print(json.dumps(game_line, indent=4, ensure_ascii=False))

            confirm = input("[您是否满意本次创作结果？(y/n)]: ").lower()
            if confirm.lower() == "y":
                return {"title": "比赛情节线", "result": game_line}
            else:
                storage.set("revision_suggestion_for_game_line", input("[请输入您的修改意见]: "))


# "主队是否有犯规":         ("bool", "根据{该时间段内比赛情节}预测主队是否会有犯规"),
# "客队是否有犯规":         ("bool", "根据{该时间段内比赛情节}预测客队是否会有犯规"),
# "主队是否有受伤":         ("bool", "根据{该时间段内比赛情节}、{客队是否有犯规}预测主队是否会有球员受伤"),
# "客队是否有受伤":         ("bool", "根据{该时间段内比赛情节}、{主队是否有犯规}预测客队是否会有球员受伤"),
# "主队是否有换人":         ("bool", "根据{该时间段内比赛情节}、{主队会做怎样的战术调整}、{主队是否有受伤}结合比赛的当前时间，一般不会在上半场就过早换人除非有受伤严重的情况，预测主队是否会有换人动作"),
# "客队是否有换人":         ("bool", "根据{该时间段内比赛情节}、{客队会做怎样的战术调整}、{客队是否有受伤}结合比赛的当前时间，一般不会在上半场就过早换人除非有受伤严重的情况，预测客队是否会有换人动作"),


In [ ]:
## 分发故事段落设计
@main_workflow.chunk()
def send_game_line_block_list(inputs, storage):

    game_line             = storage.get("game_line")
    game_line_details     = game_line["比赛情节线"]
    extra_instruction     = input("[您是否还有其他创作指导说明？如注意事项等]")
    game_line_block_list  = []  # 包括 "该时间段内比赛情节", "主队是否有战术调整", "主队会做怎样的战术调整", "客队是否有战术调整", "客队会做怎样的战术调整", "比赛激烈程度和身体对抗强度", "比赛犯规程度"

    for item in game_line_details:
        print(item)
        item.update({"补充创作指导": extra_instruction})
        game_line_block_list.append(item)

    return game_line_block_list

In [ ]:
## 最终结果整理

@main_workflow.chunk()
def sort_out(inputs, storage):
    result = []
    for item in inputs["default"]:
        result.append(item["default"])

    return "\n\n".join(result)

## 定义分块创作工作流的工作块

In [ ]:
## 获取初始数据

@block_workflow.chunk()
def init_data(inputs, storage):

    storage.set("game_line_block", inputs["default"])  # game_line_block 包括 "该时间段内比赛情节", 主队是否有战术调整, 主队会做怎样的战术调整, 客队是否有战术调整, 客队会做怎样的战术调整, 比赛激烈程度和身体对抗强度, 比赛犯规程度

    # 从公共存储中取出上一段创作结果
    storage.set("last_block_content", block_workflow.public_storage.get("last_block_content"))
    return

In [ ]:
## 过程产出输出

@block_workflow.chunk_class()
def print_process_output(inputs, storage):
    print(f"[{ inputs['default']['title'] }]:")
    if isinstance(inputs["default"]["result"], dict):
        print(json.dumps(inputs["default"]["result"], indent=4, ensure_ascii=False))
    else:
        print(inputs["default"]["result"])
    return

In [ ]:
## 进行正文创作

@block_workflow.chunk()
def generate_block_content(inputs, storage):

    game_line_block                = storage.get("game_line_block")
    match_content                  = game_line_block["该时间段内比赛情节"]
    time                           = match_content[0]["比赛时间"]

    if_tactic_change_for_home_team = game_line_block["主队是否有战术调整"]
    tactic_changes_for_home_team   = game_line_block["主队会做怎样的战术调整"]

    if_tactic_change_for_away_team = game_line_block["客队是否有战术调整"]
    tactic_changes_for_away_team   = game_line_block["客队会做怎样的战术调整"]

    match_intensity                = game_line_block["比赛激烈程度和身体对抗强度"]
    level_of_fouls                 = game_line_block["比赛犯规程度"]

    refree_name      = storage.get("refree_name")

    home_team_coach  = storage.get("home_team_coach")
    away_team_coach  = storage.get("away_team_coach")

    # Players on the pitch
    home_team_players_on_pitch = storage.get("home_team_players_on_pitch", None)
    away_team_players_on_pitch = storage.get("away_team_players_on_pitch", None)

    if not home_team_players_on_pitch:
        home_team_players_on_pitch = storage.get("home_team_line_up")

    if not away_team_players_on_pitch:
        away_team_players_on_pitch = storage.get("away_team_line_up")

    # Players on the bench
    home_team_remaining_substitutions = storage.get("home_team_remaining_substitutions", None)
    away_team_remaining_substitutions = storage.get("away_team_remaining_substitutions", None)

    if not home_team_remaining_substitutions:
        home_team_remaining_substitutions = storage.get("home_team_substitutions")

    if not away_team_remaining_substitutions:
        away_team_remaining_substitutions = storage.get("away_team_substitutions")

    # Fouls and cards
    home_team_accumulative_yellow_and_red_cards = storage.get("home_team_accumulative_yellow_and_red_cards", None)
    away_team_accumulative_yellow_and_red_cards = storage.get("away_team_accumulative_yellow_and_red_cards", None)

    # Scores
    home_team_players_scores = storage.get("home_team_players_scores", None)
    away_team_players_scores = storage.get("away_team_players_scores", None)

    ## 如果有前一段内容，通过传入前一段内容末尾确保创作的连贯性
    last_match_moments  = storage.get("last_match_moments", None)

    ## 如果有评判反馈的修改意见，添加prompt
    last_generation     = storage.get("match_moments", None)
    revision_suggestion = storage.get("revision_suggestion", None)

    # Score
    score = storage.get("score", None)

    # 2 - N 段
    if last_match_moments:

        keep_length   = 15
        block_content = (analyst_4.set_role({"角色", "你是一个职业足球比赛解说员, 能根据提供的比赛情节进行适当发挥解说比赛"})
                                  .input({"该时间段内比赛情节": match_content, "比赛时间段": time,
                                          "主队是否有战术调整": if_tactic_change_for_home_team, "主队会做怎样的战术调整": tactic_changes_for_home_team,
                                          "客队是否有战术调整": if_tactic_change_for_away_team, "客队会做怎样的战术调整": tactic_changes_for_away_team,
                                          "比赛激烈程度和身体对抗强度": match_intensity, "比赛犯规程度": level_of_fouls,
                                          "主裁判": refree_name, "主队教练": home_team_coach, "客队教练": away_team_coach,
                                          "主队场上球员": home_team_players_on_pitch, "客队场上球员": away_team_players_on_pitch,
                                          "主队未登场板凳球员": home_team_remaining_substitutions, "客队未登场板凳球员": away_team_remaining_substitutions,
                                          "主队犯规与红黄牌累计情况": home_team_accumulative_yellow_and_red_cards, "客队犯规与红黄牌累计情况": away_team_accumulative_yellow_and_red_cards,
                                          "主队球员得分情况": home_team_players_scores, "客队球员得分情况": away_team_players_scores,
                                          "比分": score, "上一段落的末尾": last_match_moments[(-1 * keep_length):]
                                           }
                                         )
                                  .instruct("""对于该{比赛时间段}内基于{该时间段内比赛情节}，考虑{主队会做怎样的战术调整}、{客队会做怎样的战术调整}、{比赛激烈程度和身体对抗强度}、{比赛犯规程度}，将{该时间段内比赛情节}扩写为解说一样的较为完整的每个比赛精彩瞬间
                                              解说中提及的球员必须是出自{主队场上球员}、{主队未登场板凳球员}、{客队场上球员}、{客队未登场板凳球员}的球员
                                              每段需要包括焦点瞬间，如在第几分钟哪位球员在哪里做了什么，比如盘带、突破、过人、传球、传中、45度传中、强弱侧转移、射门、扑救、盯防、贴身防守、对抗、铲球、犯规等等细节，结果怎样，比如盘带突破过人成功或失败，传球、传中、45度炸、转移精准找到队友或失败，射门、扑救成功或失败，谁对谁进行盯防、贴身防守、对抗、铲球、犯规，结果如何，是否被判犯规，是否被判定位球或点球
                                              根据你对{主队教练}、{客队教练}、{主裁判}的了解可以视情况加上教练和裁判的反应与举措
                                              参考{比赛犯规程度}，根据{该时间段内比赛情节}结合{主队犯规与红黄牌累计情况}、{客队犯规与红黄牌累计情况}以及你对犯规与被犯规球员的了解，判断犯规的严重性，参照你对{主裁判}的了解和他通常的判罚尺度给出判罚预测，更新{主队犯规与红黄牌累计情况}和{客队犯规与红黄牌累计情况}
                                              参考{比赛犯规程度}，根据{该时间段内比赛情节}与{主队犯规与红黄牌累计情况}，给出主队是否会换人以及谁换下谁进行预测，如有换人则更新{主队场上球员}和{主队未登场板凳球员}
                                              参考{比赛犯规程度}，根据{该时间段内比赛情节}与{客队犯规与红黄牌累计情况}，给出客队是否会换人以及谁换下谁进行预测，如有换人则更新{客队场上球员}和{客队未登场板凳球员}
                                              根据{主队会做怎样的战术调整}和你对{主队教练}的了解结合{主队未登场板凳球员}对主队是否会换人以及谁换下谁进行预测
                                              根据{客队会做怎样的战术调整}和你对{客队教练}的了解结合{客队未登场板凳球员}对客队是否会换人以及谁换下谁进行预测
                                              文末更新比分
                                              创作时需要承接{上一段落的末尾}以确保表达的连贯性""")
                                  .output({"比赛时间段":             ("str", ),
                                           "主队换人":              ("str", "主队基于战术调整或伤病进行的换人调整"),
                                           "客队换人":              ("str", "客队基于战术调整或伤病进行的换人调整"),
                                           "主队场上队员":           ("str", "主队在{主队换人}调整后的场上队员"),
                                           "客队场上队员":           ("str", "客队在{客队换人}调整后的场上队员"),
                                           "主队未登场板凳球员":      ("str", "主队在{主队换人}调整后还未被换上的替补球员"),
                                           "客队未登场板凳球员":      ("str", "客队在{客队换人}调整后还未被换上的替补球员"),
                                           "主队犯规情况":           ("str", "主队在本时段内的犯规与拿到红黄牌的情况"),
                                           "主队犯规与红黄牌累计情况": [{"球员姓名": ("str", "主队球员姓名"), "犯规情况": ("str", "该球员在该时段内的犯规"), "吃牌情况": ("str", "该球员累计吃牌情况")}],
                                           "客队犯规情况":           ("str", "客队在本时段内的犯规与拿到红黄牌的情况"),
                                           "客队犯规与红黄牌累计情况": [{"球员姓名": ("str", "客队球员姓名"), "犯规情况": ("str", "该球员在该时段内的犯规"), "吃牌情况": ("str", "该球员累计吃牌情况")}],
                                           "主队球员得分情况":        [{"球员姓名": ("str", "主队球员姓名"), "得分情况": ("str", "该球员在本时段内的进球得分情况"), "累计得分情况": ("str", "该球员的累计进球得分情况")}],
                                           "客队球员得分情况":        [{"球员姓名": ("str", "客队球员姓名"), "得分情况": ("str", "该球员在本时段内的进球得分情况"), "累计得分情况": ("str", "该球员的累计进球得分情况")}],
                                           "比赛瞬间集合":           ("str", "该时段内每个精彩瞬间集合"),
                                           "比分":                  ("str", "该时段结束时比分")
                                            }
                                           )
                                  .start()
                                )

        storage.set("home_team_players_on_pitch", block_content["主队场上队员"])
        storage.set("away_team_players_on_pitch", block_content["客队场上队员"])
        storage.set("home_team_remaining_substitutions", block_content["主队未登场板凳球员"])
        storage.set("away_team_remaining_substitutions", block_content["客队未登场板凳球员"])
        storage.set("home_team_accumulative_yellow_and_red_cards", block_content["主队犯规与红黄牌累计情况"])
        storage.set("away_team_accumulative_yellow_and_red_cards", block_content["客队犯规与红黄牌累计情况"])
        storage.set("home_team_players_scores", block_content["主队球员得分情况"])
        storage.set("away_team_players_scores", block_content["客队球员得分情况"])
        storage.set("match_moments", block_content["比赛瞬间集合"])
        storage.set("score", block_content["比分"])

        return {"title":  f'时间段: {block_content["比赛时间段"]}',
                "result": block_content["比赛瞬间集合"]}

    # 修改
    elif last_generation and revision_suggestion:

        updated_block_content = (analyst_4.set_role({"角色", "你是一个职业足球比赛解说员, 能根据提供的比赛情节进行适当发挥解说比赛"})
                                          .input({"已有创作结果": last_generation, "修改意见": revision_suggestion})
                                          .instruct("""你之前已经创作了{已有创作结果}，但仍然需要修改，请参考{修改意见}进行修订""")
                                          .start()
                                    )

        storage.set("match_moments", updated_block_content["比赛瞬间集合"])

        return {"title":  f'时间段: {updated_block_content["比赛时间段"]}',
                "result": updated_block_content["比赛瞬间集合"]}

    # 第一段
    else:

        block_content = (analyst_4.set_role({"角色", "你是一个职业足球比赛解说员, 能根据提供的比赛情节进行适当发挥解说比赛"})
                                  .input({"该时间段内比赛情节": match_content, "比赛时间段": time,
                                          "主队是否有战术调整": if_tactic_change_for_home_team, "主队会做怎样的战术调整": tactic_changes_for_home_team,
                                          "客队是否有战术调整": if_tactic_change_for_away_team, "客队会做怎样的战术调整": tactic_changes_for_away_team,
                                          "比赛激烈程度和身体对抗强度": match_intensity, "比赛犯规程度": level_of_fouls,
                                          "主裁判": refree_name, "主队教练": home_team_coach, "客队教练": away_team_coach,
                                          "主队场上球员": home_team_players_on_pitch, "客队场上球员": away_team_players_on_pitch,
                                          "主队未登场板凳球员": home_team_remaining_substitutions, "客队未登场板凳球员": away_team_remaining_substitutions
                                          }
                                         )
                                  .instruct("""对于该{比赛时间段}内基于{该时间段内比赛情节}，考虑{主队会做怎样的战术调整}、{客队会做怎样的战术调整}、{比赛激烈程度和身体对抗强度}、{比赛犯规程度}，将{该时间段内比赛情节}扩写为解说一样的较为完整的每个比赛精彩瞬间
                                              解说中提及的球员必须是出自{主队场上球员}、{主队未登场板凳球员}、{客队场上球员}、{客队未登场板凳球员}的球员
                                              每段需要包括焦点瞬间，如在第几分钟哪位球员在哪里做了什么，比如盘带、突破、过人、传球、传中、45度传中、强弱侧转移、射门、扑救、盯防、贴身防守、对抗、铲球、犯规等等细节，结果怎样，比如盘带突破过人成功或失败，传球、传中、45度炸、转移精准找到队友或失败，射门、扑救成功或失败，谁对谁进行盯防、贴身防守、对抗、铲球、犯规，结果如何，是否被判犯规，是否被判定位球或点球
                                              根据你对{主队教练}、{客队教练}、{主裁判}的了解可以视情况加上教练和裁判的反应与举措
                                              参考{比赛犯规程度}，根据{该时间段内比赛情节}结合{主队犯规与红黄牌累计情况}、{客队犯规与红黄牌累计情况}以及你对犯规与被犯规球员的了解，判断犯规的严重性，参照你对{主裁判}的了解和他通常的判罚尺度给出判罚预测，更新{主队犯规与红黄牌累计情况}和{客队犯规与红黄牌累计情况}
                                              参考{比赛犯规程度}，根据{该时间段内比赛情节}与{主队犯规与红黄牌累计情况}，给出主队是否会换人以及谁换下谁进行预测，如有换人则更新{主队场上球员}和{主队未登场板凳球员}
                                              参考{比赛犯规程度}，根据{该时间段内比赛情节}与{客队犯规与红黄牌累计情况}，给出客队是否会换人以及谁换下谁进行预测，如有换人则更新{客队场上球员}和{客队未登场板凳球员}
                                              根据{主队会做怎样的战术调整}和你对{主队教练}的了解结合{主队未登场板凳球员}对主队是否会换人以及谁换下谁进行预测
                                              根据{客队会做怎样的战术调整}和你对{客队教练}的了解结合{客队未登场板凳球员}对客队是否会换人以及谁换下谁进行预测
                                              文末更新比分""")
                                  .output({"比赛时间段":            ("str", ),
                                           "主队换人":              ("str", "主队基于战术调整或伤病进行的换人调整"),
                                           "客队换人":              ("str", "客队基于战术调整或伤病进行的换人调整"),
                                           "主队场上队员":           ("str", "主队在{主队换人}调整后的场上队员"),
                                           "客队场上队员":           ("str", "客队在{客队换人}调整后的场上队员"),
                                           "主队未登场板凳球员":      ("str", "主队在{主队换人}调整后还未被换上的替补球员"),
                                           "客队未登场板凳球员":      ("str", "客队在{客队换人}调整后还未被换上的替补球员"),
                                           "主队犯规情况":           ("str", "主队在本时段内的犯规与拿到红黄牌的情况"),
                                           "主队犯规与红黄牌累计情况": [{"球员姓名": ("str", "主队球员姓名"), "犯规情况": ("str", "该球员在该时段内的犯规"), "吃牌情况": ("str", "该球员累计吃牌情况")}],
                                           "客队犯规情况":           ("str", "客队在本时段内的犯规与拿到红黄牌的情况"),
                                           "客队犯规与红黄牌累计情况": [{"球员姓名": ("str", "客队球员姓名"), "犯规情况": ("str", "该球员在该时段内的犯规"), "吃牌情况": ("str", "该球员累计吃牌情况")}],
                                           "主队球员得分情况":        [{"球员姓名": ("str", "主队球员姓名"), "得分情况": ("str", "该球员在本时段内的进球得分情况"), "累计得分情况": ("str", "该球员的累计进球得分情况")}],
                                           "客队球员得分情况":        [{"球员姓名": ("str", "客队球员姓名"), "得分情况": ("str", "该球员在本时段内的进球得分情况"), "累计得分情况": ("str", "该球员的累计进球得分情况")}],
                                           "比赛瞬间集合":           ("str", "该时段内每个精彩瞬间集合"),
                                           "比分":                  ("str", "该时段结束时比分")
                                            }
                                           )
                                  .start()
                        )

        storage.set("home_team_players_on_pitch", block_content["主队场上队员"])
        storage.set("away_team_players_on_pitch", block_content["客队场上队员"])
        storage.set("home_team_remaining_substitutions", block_content["主队未登场板凳球员"])
        storage.set("away_team_remaining_substitutions", block_content["客队未登场板凳球员"])
        storage.set("home_team_accumulative_yellow_and_red_cards", block_content["主队犯规与红黄牌累计情况"])
        storage.set("away_team_accumulative_yellow_and_red_cards", block_content["客队犯规与红黄牌累计情况"])
        storage.set("home_team_players_scores", block_content["主队球员得分情况"])
        storage.set("away_team_players_scores", block_content["客队球员得分情况"])
        storage.set("match_moments", block_content["比赛瞬间集合"])
        storage.set("score", block_content["比分"])

        return {"title":  f'时间段: {block_content["比赛时间段"]}',
                "result": block_content["比赛瞬间集合"]}

In [ ]:
## 人类判断是否满意

@block_workflow.chunk()
def human_confirm(inputs, storage):

    confirm = ""
    while confirm.lower() not in ("y", "n"):
        confirm = input("[您是否满意本次创作结果？(y/n)]: ")
    return confirm.lower()

In [ ]:
## 提交修改意见

@block_workflow.chunk()
def input_revision_suggestion(inputs, storage):

    storage.set("revision_suggestion", input("[请输入您的修改意见]: "))
    return

In [ ]:
## 输出满意的创作成果

@block_workflow.chunk()
def return_block_content(inputs, storage):
    match_moments = storage.get("match_moments")

    # 记得在公共存储中更新本次创作结果
    block_workflow.public_storage.set("last_match_moments", match_moments)
    return match_moments

## 定义分块创作工作流的工作流程

In [ ]:

(
    block_workflow
       .connect_to("init_data")
       .connect_to("generate_block_content")
       .connect_to("@print_process_output")
       .connect_to("human_confirm")
       .if_condition(lambda return_value, storage: return_value == "y")
            .connect_to("return_block_content")
            .connect_to("end")
       .else_condition()
            .connect_to("input_revision_suggestion")
            .connect_to("generate_block_content")
)


(
    main_workflow.chunks["START"]
       .connect_to("input_match_basic_info")
       .connect_to("generate_game_scenario")
       .connect_to("generate_player_performance")
       .connect_to("generate_game_line")
       .connect_to("send_game_line_block_list")
           .loop_with(block_workflow)
       .connect_to("END")
  )

In [ ]:
main_workflow.draw(type="img")

In [ ]:
# 开始执行
result = main_workflow.start()
print(result["default"])

[💡请输入主裁判名字]:大卫奥利弗
[💡请输入赛事名称]:英格兰足球超级联赛
[💡请输入主场球队]:阿森纳
[💡请输入客场球队]:利物浦
[💡请输入天气情况]:晴天
[💡请输入主场球队教练]:阿尔特塔
[💡请输入客场球队教练]:阿尔恩-斯洛特
[💡请输入主场球队首发11人名单]:大卫拉亚、卡拉菲奥里、加布里埃尔、萨利巴、廷贝尔、赖斯、托马斯帕泰、厄德高、马丁内利、哈弗茨、萨卡
[💡请输入主场球队替补队员名单]:拉希姆斯特林，热苏斯、富安健洋、蒂尔尼、若日尼奥、基维奥尔、恩瓦内里，米克尔梅里诺，津琴科
[💡请输入主场球队阵型]:4-3-3
[💡请输入客场球队首发11人名单]:阿利森、罗伯逊、范戴克、科纳特、阿诺德、麦卡利斯特、赫拉芬贝赫、路易斯迪亚斯、索博斯洛伊、萨拉赫、达尔文努涅斯
[💡请输入客场球队替补队员名单]:琼斯，加克波，乔-戈麦斯，凯莱赫，科纳布拉德利，远藤航，宽萨，费德里科-基耶萨，埃利奥特
[💡请输入客场球队阵型]:4-2-3-1
[💡请输入主场球队战术风格]:实用性的地面进攻加控球和一定程度的高位压迫，注重角球与定位球在进攻中的作用，防守时注重身体对抗与强度
[💡请输入客场球队战术风格]: 一定程度沿袭了克洛普时期的压迫打法，跑动积极，有强调中场洗球，防守时也倚赖范戴克为首的个人能力
[💡请输入主场球队近期战绩]:英超目前4胜2平0负，进12球失5球
[💡请输入客场球队近期战绩]:英超目前5胜0平1负，进12球失2球
[💡请输入主场球队球员本赛季数据]:热苏斯英超出场4次，其中首发0次替补4次，0球0助3抢断3犯规2黄牌。斯特林英超出场2次，其中首发0次替补2次，0球0助。哈弗茨英超出场6次，其中首发6次, 3球1助5抢断11犯规0黄牌。萨卡英超出场6次，其中首发6次，1球5助10抢断3犯规1黄牌。马丁内利英超出场6次，其中首发5次替补1次，1球2助5抢断4犯规0黄牌。恩瓦内里英超出场2次，其中首发0次替补2次，0球0助。托马斯帕泰英超出场6次，其中首发6次,1球0助17抢断10犯规2黄牌。赖斯英超出场5次，其中首发5次，0球0助2抢断4犯规2黄牌1红牌。梅里诺英超出场0次。若日尼奥英超出场1次，其中首发1次，0球0助1抢断2犯规1黄牌。厄德高英超出场3次，其中首发3次，0球0助3抢断1犯规1黄牌。津琴科英超出场2次，其中首发1次替补1次，0球0助0